In [1]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver


def get_url(search_term):
    template = 'https://www.amazon.in/s?k={}&ref=nb_sb_noss'
    search_term = search_term.replace(' ','+')
    url = template.format(search_term)
    url = url + '&page{}'
    return url


def extract_record(item):
    antag = item.h2.a
    description = antag.text.strip()
    url = 'https://www.amazon.com' + antag.get('href')
    
    # price
    try:
        price_parent = item.find('span','a-price')
        price = price_parent.find('span','a-offscreen').text
        
    except AttributeError:
        return 

    
    try:
        # ratings
        rating = item.i.text

        # review_count
        review_count = item.find('span', {'class': 'a-size-base s-underline-text'}).text
    
    except AttributeError:
        rating = ''
        review_count = ''
        
        
    
    # storing all in a tuple
    result = (description , price , rating , review_count , url)
    
    return result
    

    
def main(search_term):        
    driver = webdriver.Chrome()   
    records = []   
    url = get_url(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()

    with open('results.csv','w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description' , 'Price' , 'Rating' , 'ReviewCount' , 'Url'])
        writer.writerows(records)
      

In [ ]:
main('bags')